In [ ]:
# import comet_ml in the top of your file
from comet_ml import Experiment
# Add the following code anywhere in your machine learning file
experiment = Experiment(api_key="Go4XzK6f4G10W9jFaWiHg6JP2",
                        project_name="tbm", workspace="twofyw")
record = True

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
config = Config('rnn-normalized-full-30-s-0-1', debug=False, sl=30, cont_names=Config.all_columns(), postpond=0, fn_feather='tbmData/feather/all_columns-full-30-s-0.feather')

torch.cuda.set_device(config.gpu_start); torch.ones((1, 1)).cuda(); # wakeup gpu

# load data
cyc_cont = pd.read_feather(config.fn_feather)
npz = np.load(config.fn_np)
idx, stat_x, stat_y = npz['idx'], npz['stat_x'], npz['stat_y']

rnndb = MultiDeptTabularDataBunch.from_df('tmp', cyc_cont, config.dep_var, valid_idx=config.valid_idx_tile, bs=config.bs, num_workers=0)

record = False

In [ ]:
hyper_params = {
    'n_hidden': 100,
    'n_layers': 3,
    'hidden_p': 0.3,
    'input_p': 0.6,
    'weight_p': 0.5,
    'layers': [3*config.n_cont, 5, 2],
    'drops': [0.2] * 3,
    'alpha': 2.,
    'beta': 1.,
    'clip': 0.5,
#     'loss_func' = MSELossFlat()
    'loss_func': l1
}
if record:
    experiment.log_parameters(hyper_params)

rnn_enc = ContModel1(1, config.n_cont, hyper_params['n_hidden'], hyper_params['n_layers'], sl=config.sl,
                     hidden_p=hyper_params['hidden_p'], input_p=hyper_params['input_p'], embed_p=0, weight_p=hyper_params['weight_p'])

model = MultiInputSequentialRNN(rnn_enc, PoolingLinearClassifier(hyper_params['layers'], hyper_params['drops'])).cuda()


metrics = MAPD(stat_y)
learner = Learner(rnndb, model, loss_func=hyper_params['loss_func'], metrics=metrics, opt_func=optim.SGD, )

from fastai.callbacks import *
# learner.callback_fns += [ShowGraph, partial(SaveModelCallback, name='rnn0')]
learner.callback_fns += [ShowGraph,]
learner.callbacks += [TerminateOnNaNCallback()]
learner.callbacks.append(RNNTrainer(learner, config.sl, alpha=hyper_params['alpha'], beta=hyper_params['beta']))

learner.callback_fns.append(partial(GradientClipping, clip=hyper_params['clip']))
learner.split(rnn_classifier_split)

In [ ]:
learner.unfreeze()
learner.lr_find(num_it=100)
learner.recorder.plot(1,1)

In [ ]:
with experiment.train():
    lr = 1e-1
    learner.fit_one_cycle(100, lr)

In [ ]:
# with experiment.train():
lr = 1e-1
learner.fit_one_cycle(100, lr)

In [ ]:
# with experiment.train():
lr = 1e-2
learner.fit_one_cycle(100, lr)

In [ ]:
# with experiment.train():
lr = 1e-1
learner.fit_one_cycle(100, lr)

In [ ]:
with experiment.train():
    lr = 1e-1
    learner.fit_one_cycle(100, lr)

In [ ]:
experiment.end()

In [ ]:
learner.model.eval();

In [ ]:
apply_leaf(learner.model, lambda m: m.train() if m.__module__.endswith('rnn') else None)
# learner.model = learner.model.train()

In [ ]:
# it = iter(learner.data.fix_dl)
# it = iter(learner.data.train_dl)
it = iter(learner.data.valid_dl)

In [ ]:
# x, y = ni(learner.data.train_dl)
# x, y = ni(learner.data.valid_dl)
x, y = next(it)
x[1] = x[1].requires_grad_()
learner.model.reset()
# y, p = y.cpu(), learner.model(*x)[0].exp().cpu()
y, p = y.cpu(), learner.model(*x)[0].cpu()
# y[:,0], p[:,0] = 1,1
y_np, p_np = to_np(y), to_np(p)
y_denormed, p_denormed = denormalize(y_np, *stat_y), denormalize(p_np, *stat_y)

mapd_denorm = MAPD(stat_y)
l_np = mapd_denorm(p_denormed, y_denormed); l_np
# l = mapd(p, y); l
# experiment.log_metric('denormed_mapd', )

In [ ]:
experiment.log_metric('mapd', l_np)

In [ ]:
??learner.pred_batch

In [ ]:
learner.save(config.exp_name)

In [ ]:
i = 0
plt.scatter(y_denormed[:,i], p_denormed[:,i]);

In [ ]:
i = 1
plt.scatter(y_denormed[:,i], p_denormed[:,i]);

### Feature importance

In [ ]:
d = torch.autograd.grad(l, x[1], retain_graph=True)[0].mean(0)
d_np = np.abs(to_np(d))

Last time step

In [ ]:
# last timestep
i, c, t = 0, 1, 29
d_reshape = d[t*config.n_cont:(t+1)*config.n_cont]
argsort = d_reshape.argsort()

In [ ]:
np.array(config.cont_names)[argsort]

All time steps

In [ ]:
d_reshape = d_np.reshape(config.sl, config.n_cont).mean(0)
argsort = d_reshape.argsort()[::-1]
list(zip(np.array(config.cont_names)[argsort], d_reshape[argsort]))

Plot the most important columns

### X check

In [ ]:
s = 2
x_np = to_np(x[1][s*config.mulr]).reshape(config.sl, config.n_cont)
x_np.shape
for feat in range_of(x_np[0]):
#     plt.figure()
    ax = plt.plot(x_np[:,feat])
#     legend = plt.legend(config.cont_names[feat])
#     set_font(legend.texts)

In [ ]:
ax = cycles.loc[s,config.cont_names].iloc[idx[s]:idx[s]+config.sl].plot()
set_font(ax.legend().texts)

In [ ]:
# last timestep
i, c, t = 0, 1, 29
d = torch.autograd.grad(l, x[1], retain_graph=True)
d = d[0][i,t*n_cont:(t+1)*n_cont]
d_np = to_np(d)

In [ ]:
# average along time axis
i, c = 3, 1
d = torch.autograd.grad(p[i,c], x[1], retain_graph=True)
d = d[0][i].view(n_cont, -1)
d_np = to_np(d)
d_np = np.abs(d_np)
d_np = d_np.mean(1)

In [ ]:
impact = d_np.argsort()[::-1]
cs = columns_x[impact]
pprint(list(enumerate(zip(impact, d_np[impact], cs))))
plots(cycles[i].loc[:,cs[:10]].iloc[idx[i]-50:idx[i]+100]);

In [ ]:
# l x b x t x h
l, b = 0, 0
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for t in range(30)]

In [ ]:
# l x b x t x h
l, b, t = 0, 0, 1
[(float(p[l][b][t].mean()), float(p[l][b][t].std())) for l in range(n_layers)]

In [ ]:
list(zip(y_np.tolist(), p_np.tolist()))

In [ ]:
i = 0
plt.scatter(_s[:,i], y_s[:,i])

In [ ]:
i = 1
plt.scatter(p_s[:,i], y_s[:,i])